# Installing packages and imports

In [ ]:
!pip install aiohttp
!pip install nest-asyncio

In [11]:
import requests
import aiohttp
import asyncio
import nest_asyncio
import os
import json
import pandas as pd

# User authentication

Modify the below code to include your details. This is required for authentication.

In [1]:
email = 'example@email.com'
password = 'password'

# Getting the shot IDs

In [ ]:
def get_private_shot_ids(auth):
    url = 'https://visualizer.coffee/api/shots/'
    params = {
        'page': 1,
        'items': 100
    }

    ids = []  # Array to store the IDs

    while True:
        response = requests.get(url, params=params, auth=auth)

        if response.status_code == 200:
            shots_data = response.json()
            shots = shots_data['data']

            if len(shots) == 0:
                break  # Break the loop if there is no data

            # Extract and store the IDs
            for shot in shots:
                ids.append(shot['id'])

            # Update the page number for the next request
            params['page'] += 1

        else:
            print(f"Request failed with status code {response.status_code}")
            break

    return ids

auth = (email, password)
shot_ids = get_private_shot_ids(auth)

# The number of shots printed shuold match the number
# on your visualizer.coffee/shots
print(f"Total number of IDs: {len(shot_ids)}")


# Using the IDs to get the shot data

In [6]:

async def get_shot_data(session, shot_id):
    # Construct the URL using the shot_id
    url = f'https://visualizer.coffee/api/shots/{shot_id}/download?essentials='

    # Send an HTTP GET request to the URL using the session
    async with session.get(url) as response:
        # Check if the request was successful (status code 200)
        if response.status == 200:
            # If successful, extract the JSON data from the response
            shot_data = await response.json()
            return shot_data
        else:
            # If the request failed, print an error message with the status code
            print(f"Request failed with status code {response.status}")
            return None

async def get_all_shot_data(shot_ids):
    # Create a new HTTP session using aiohttp
    async with aiohttp.ClientSession() as session:
        # Create a list of tasks to fetch shot data for each shot_id
        tasks = [get_shot_data(session, shot_id) for shot_id in shot_ids]

        # Use asyncio.gather to run all the tasks concurrently
        shots_data = await asyncio.gather(*tasks)

    # Return the list of shot data
    return shots_data


In [7]:
# This allows running asyncio in a Jupyter Notebook or similar environment
nest_asyncio.apply()

# Execute the get_all_shot_data function and store the result in shots_data
shots_data = asyncio.run(get_all_shot_data(shot_ids))

# Downloading each shot as a json file

In [12]:
def download_shots_as_json(shots_data):
    # Create a new folder called "shots" in the current directory
    os.makedirs("shots", exist_ok=True)

    # Iterate over the shots_data array and save each element as a JSON file
    for shot_data in shots_data:
        # Extract the profile ID from the shot_data
        profile_id = shot_data["id"]

        # Generate the file path for the JSON file
        file_path = f"shots/{profile_id}.json"

        # Write the shot_data as JSON to the file
        with open(file_path, "w") as file:
            json.dump(shot_data, file)

# Execute the get_all_shot_data function and store the result in shots_data
shots_data = asyncio.run(get_all_shot_data(shot_ids))

# Call the download_shots_as_json function to save the data as JSON files
download_shots_as_json(shots_data)